# A two stage stochastic program

Assume that you are warming your house with fuel. Currently fuel is at 750€/1000 liters. 
You can store up to 1000l for the winter, but you don't know yet how much you will use, and how much it will cost to buy fuel during winter.

We assume that there are $4$ scenarios, with probabilities $p$, futur fuel cost $c$ and demand $d$ given afterwards.
We want to minimize the expected costs of warming yourself during winter.

In [2]:
using JuMP, Clp

S = 4 
p = [0.1 0.4 0.4 0.1]
c0 = 750
c1 = [700 770 900 900]
d = [500 700 800 1000]

1×4 Array{Int64,2}:
 500  700  800  1000

## Question 1

Solve the corresponding 2 stage program. Give the optimal value, and optimal controls.

In [14]:
m_2S = Model(solver = ClpSolver())


    @variable(m_2S,u0>=0)
    @variable(m_2S,u1[1:4])

    @objective(m_2S, Min, (c0*u0 + sum(p[i]*u1[i]*c1[i] for i=1:4))/1000)
    
    @constraint(m_2S, u0 >=0)
    @constraint(m_2S, u0 <= 1000)
    for i=1:4
        @constraint(m_2S, u1[i] >=0)
    @constraint(m_2S, u1[i] <=1000-u0)
        @constraint(m_2S, u1[i] >=d[i]-u0)
            end
    
     solve(m_2S)
#print(m_2S)
println(getobjectivevalue(m_2S))
println("u0 = ",getvalue(u0))
println("u1 = ",getvalue(u1))

Min 0.75 u0 + 0.07 u1[1] + 0.308 u1[2] + 0.36 u1[3] + 0.09 u1[4]
Subject to
 u0 ≥ 0
 u0 ≤ 1000
 u1[1] ≥ 0
 u1[1] + u0 ≤ 1000
 u1[1] + u0 ≥ 500
 u1[2] ≥ 0
 u1[2] + u0 ≤ 1000
 u1[2] + u0 ≥ 700
 u1[3] ≥ 0
 u1[3] + u0 ≤ 1000
 u1[3] + u0 ≥ 800
 u1[4] ≥ 0
 u1[4] + u0 ≤ 1000
 u1[4] + u0 ≥ 1000
 u1[i] ∀ i ∈ {1,2,3,4}
 u0 ≥ 0
588.0
u0 = 700.0
u1_1 = -0.0
u1_2 = -0.0
u1_3 = 100.0
u1_4 = 300.0


## Question 2

Solve the open-loop version of this problem. Give the optimal value, and optimal controls.
Check the inequality between the value of the open-loop problem and 2-stage problem.

In [15]:
m_OL = Model(solver = ClpSolver())

    @variable(m_OL,u0>=0)
    @variable(m_OL,u1)

    @objective(m_OL, Min, (c0*u0 + sum(p[i]*u1*c1[i] for i=1:4))/1000)
    
    @constraint(m_OL, u0 >=0)
    @constraint(m_OL, u0 <= 1000)
       @constraint(m_OL, u1 >=0)
    @constraint(m_OL, u1 <=1000-u0)
    for i=1:4
        @constraint(m_OL, u1 >=d[i]-u0)
            end
    
     solve(m_OL)
#print(m_OL)
println(getobjectivevalue(m_OL))
println("u0 = ",getvalue(u0))
println("u1 = ",getvalue(u1))

Min 0.75 u0 + 0.828 u1
Subject to
 u0 ≥ 0
 u0 ≤ 1000
 u1 ≥ 0
 u1 + u0 ≤ 1000
 u1 + u0 ≥ 500
 u1 + u0 ≥ 700
 u1 + u0 ≥ 800
 u1 + u0 ≥ 1000
 u0 ≥ 0
 u1
750.0
u0 = 1000.0
u1 = -0.0


## Question 3

Solve the anticipative version of this problem. Give the optimal value, and optimal controls. Check the inequality between the value of the anticipative problem and 2-stage problem.

In [20]:
m_a = Model(solver = ClpSolver())
    @variable(m_a,u0[1:4])
    @variable(m_a,u1[1:4])

    @objective(m_a, Min, (sum(p[i]*(c0*u0[i] + u1[i]*c1[i]) for i=1:4))/1000)
    
    for i=1:4
    @constraint(m_a, u0[i] >=0)
    @constraint(m_a, u0[i] <= 1000)
        @constraint(m_a, u1[i] >=0)
    @constraint(m_a, u1[i] <=1000-u0[i])
        @constraint(m_a, u1[i] >=d[i]-u0[i])
            end
    
     solve(m_a)
#print(m_a)
println(getobjectivevalue(m_a))
println("u0 = ",getvalue(u0))
println("u1 = ",getvalue(u1))

Min 0.075 u0[1] + 0.07 u1[1] + 0.3 u0[2] + 0.308 u1[2] + 0.3 u0[3] + 0.36 u1[3] + 0.075 u0[4] + 0.09 u1[4]
Subject to
 u0[1] ≥ 0
 u0[1] ≤ 1000
 u1[1] ≥ 0
 u1[1] + u0[1] ≤ 1000
 u1[1] + u0[1] ≥ 500
 u0[2] ≥ 0
 u0[2] ≤ 1000
 u1[2] ≥ 0
 u1[2] + u0[2] ≤ 1000
 u1[2] + u0[2] ≥ 700
 u0[3] ≥ 0
 u0[3] ≤ 1000
 u1[3] ≥ 0
 u1[3] + u0[3] ≤ 1000
 u1[3] + u0[3] ≥ 800
 u0[4] ≥ 0
 u0[4] ≤ 1000
 u1[4] ≥ 0
 u1[4] + u0[4] ≤ 1000
 u1[4] + u0[4] ≥ 1000
 u0[i] ∀ i ∈ {1,2,3,4}
 u1[i] ∀ i ∈ {1,2,3,4}
560.0
u0 = [-0.0, 700.0, 800.0, 1000.0]
u1 = [500.0, -0.0, -0.0, -0.0]


## Question 4

We call (P_mean) the problem where cost and demand are replaced by their expectation. Solve this problem, giving value and first and second stage optimal control.

Evaluate the value of this first stage control, that is the expected cost of using this first stage control with adapted recourse.

Compare both values to the precedents problems.

In [10]:
m_mean = Model(solver = ClpSolver())
    @variable(m_mean,u0)
    @variable(m_mean,u1)

    @objective(m_mean, Min, (c0*u0 + u1 * sum(p[i]*c1[i] for i=1:4))/1000)

    @constraint(m_mean, u0 >=0)
    @constraint(m_mean, u0 <= 1000)
        @constraint(m_mean, u1 >=0)
    @constraint(m_mean, u1 <=1000-u0)
        @constraint(m_mean, u1 >=(sum(d[i]/4 for i=1:4))-u0)
            
    
     solve(m_mean)
#print(m_mean)
println(getobjectivevalue(m_mean))
println("u0 = ",getvalue(u0))
println("u1 = ",getvalue(u1))

562.5
u0 = 750.0
u1 = -0.0
